# Zweifeldträger Schulbeispiel

Das Vorlesungsbeispiel verfolgt das Ziel, die Verformungen eines Zweifeldträgers für zwei Betonstähle mit unterschiedlichen Duktilitätsklassen aufzuzeigen. @jager_stahlbeton_2009.

![Statisches System des Zweifeldträgers mit Abmessungen versehen, daneben der Querschnitt mit Angaben zu der Längsbewehrung](../imgs/zweifeld_system.png){#fig-zweifeld_system}

## Annahmen

- Zwei unterschiedliche Stähle
- konstant gerissene Biegesteifigkeit entlang der Stabachse

In [1]:
from packages import *

In [2]:
diam_x = 14 * mm  # Längsbewehrung in X-Richtung gerichtet
s_x = 200 * mm  # Teilung der Längsbewehrung
diam_y = 12 * mm  # Längsbewehrung in Y-Richtung gerichtet
s_y = 200 * mm  # Teilung der Längsbewehrung
c = 20 * mm  # Betonüberdeckung
h = 300 * mm  # Plattenstärke

d_x = h - c - diam_y - 1 / 2 * diam_x  # Statische Höhe in X-Richtung

put_out()

$$
\begin{aligned}
\oslash_{x} & = 14 \ \mathrm{mm} \quad & s_{x} & = 200 \ \mathrm{mm} \quad & \oslash_{y} & = 12 \ \mathrm{mm} \\ 
s_{y} & = 200 \ \mathrm{mm} \quad & c & = 20 \ \mathrm{mm} \quad & h & = 300 \ \mathrm{mm} \\ 
d_{x} & = 261.0 \ \mathrm{mm} \quad &  \quad &  
 \end{aligned}
$$

## Kaltverformter Betonstahl B500A



In [3]:
f_sy = 570 * N / mm**2  # Fliessgrenze
f_su = 600 * N / mm**2  # Zugfestigkeit
epsilon_sy = 2.78 / 1000  # Fliessdehnung
epsilon_su = 25 / 1000  # Bruchdehnung
E_s = 205 * kN / mm**2  # Elastizitätsmodul
E_sh = 1350 * N / mm**2  # Verfestigungsmodul

put_out(precision=4)

$$
\begin{aligned}
f_{sy} & = 570.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{su} & = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \epsilon_{sy} & = 0.0028 \\ 
\epsilon_{su} & = 0.025 \quad & E_{s} & = 205.0 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \quad & E_{sh} & = 1350.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

### Gerissene Biegesteifigkeit